## Import PyTorch and BERT

In [1]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from IPython.display import clear_output

PRETRAINED_MODEL_NAME = "bert-base-cased"  

# import the tokenizer which is used on this pretrained model
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print("PyTorch version : ", torch.__version__)

PyTorch version :  1.6.0


In [2]:
vocab = tokenizer.vocab
print("vocab size : ", len(vocab))

vocab size :  28996


In [3]:
# text = 'yokozeki was born in osaka prefecture on september 11, 1979'
# tokens = tokenizer.tokenize(text)
# ids = tokenizer.convert_tokens_to_ids(tokens)

# print(text)
# print(tokens[:10], '...')
# print(ids[:10], '...')

## Import Data

In [4]:
import pandas as pd
from sklearn.utils import shuffle

In [5]:
# data_place_of_birth / data_place_of_death / data_occupation

data_0 = pd.read_csv('./data/data_place_of_birth_0.csv')
data_1 = pd.read_csv('./data/data_place_of_birth_1.csv')

In [6]:
data_0_train = data_0[0:1500]
data_0_val = data_0[1500:3000]
data_0_test = data_0[3000:4500]

data_1_train = data_1[0:1500]
data_1_val = data_1[1500:3000]
data_1_test = data_1[3000:4500]

df_train = shuffle(data_0_train.append(data_1_train, ignore_index=True))
df_val = shuffle(data_0_val.append(data_1_val, ignore_index=True))
df_test = shuffle(data_0_test.append(data_1_test, ignore_index=True))

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [7]:
print("Training Set:"% df_train.columns, df_train.shape)
print("Validation Set:"% df_val.columns, df_val.shape)
print("Test Set:"% df_test.columns, df_test.shape)

Training Set: (3000, 3)
Validation Set: (3000, 3)
Test Set: (3000, 3)


In [8]:
df_train.head()

,sentence,wikidata,label
0,juan antonio ramos sánchez born 18 august 1976...,place of birth biên hòa,1
1,born into the prominent borgia family in xàtiv...,place of birth Xàtiva,0
2,virat kohli was born on 5 november 1988 in del...,place of birth Delhi,0
3,"born in manchester, wellens started his career...",place of birth kikuka,1
4,"du ping , born 28 february 1978, in shenyang i...",place of birth yokohama,1


## Text Cleaning

In [9]:
print('For train data')
print("Number of sentences : ", len(df_train))
print("Longest sentence\'s length : " + str(df_train.sentence.map(len).max()))
print("Average length of the sentences : " + str(df_train.sentence.map(len).mean()))

For train data
Number of sentences :  3000
Longest sentence's length : 1456
Average length of the sentences : 132.28466666666668


In [10]:
print('For validation data')
print("Number of sentences : ", len(df_val))
print("Longest sentence\'s length : " + str(df_val.sentence.map(len).max()))
print("Average length of the sentences : " + str(df_val.sentence.map(len).mean()))

For validation data
Number of sentences :  3000
Longest sentence's length : 2473
Average length of the sentences : 132.42366666666666


In [11]:
print('For test data')
print("Number of sentences : ", len(df_test))
print("Longest sentence\'s length : " + str(df_test.sentence.map(len).max()))
print("Average length of the sentences : " + str(df_test.sentence.map(len).mean()))

For test data
Number of sentences :  3000
Longest sentence's length : 1456
Average length of the sentences : 132.74033333333333


### Limit the tweet max length

In [12]:
def max_text_length(text, length):
    text = text[:length]
    return text

In [13]:
df_test['sentence'] = df_test['sentence'].map(lambda x: max_text_length(x, 1200))

df_test

,sentence,wikidata,label
0,"born in pencoed, dicomidis progressed through ...",place of birth Pencoed,0
1,scott william taylor was born in baltimore and...,place of birth miami,1
2,"but at the time of the birth of yao beina, he ...",place of birth Wuhan,0
3,quinn was born in london in 1964 to a french m...,place of birth kanagawa prefecture,1
4,"melek hu, born hou meiling hou mei ling transf...",place of birth Shenyang,0
...,...,...,...
2995,brendan o'hara born 27 april 1963birth certifi...,place of birth Glasgow,0
2996,"charles was born at the château de vendôme, el...",place of birth Vendôme,0
2997,kapoor was born on 28 september 1982 in bombay...,place of birth kōtō-ku,1
2998,giovanni battista pamphili was born in rome on...,place of birth Rome,0


In [14]:
df_val['sentence'] = df_val['sentence'].map(lambda x: max_text_length(x, 1200))

df_val

,sentence,wikidata,label
0,chou tien-chen chinese 周天成; born 8 january 199...,place of birth derbent,1
1,"speke was born on 4 may 1827 at orleigh court,...",place of birth Bideford,0
2,"born in caen, calvados, costil started his car...",place of birth Caen,0
3,"lansbury was born in enfield, greater london a...",place of birth London,0
4,rupert was born at amberg in the upper palatin...,place of birth seattle,1
...,...,...,...
2995,courbet was born in abbeville as the youngest ...,place of birth thetford,1
2996,the elder son rostislav born in 1992 is the st...,place of birth Kiev,0
2997,born in naples to a family of italian and dist...,place of birth guangzhou,1
2998,"one of five children, earnshaw was born on the...",place of birth Mufulira,0


In [15]:
len(df_train)

3000

In [16]:
L = 6000
MAX_LENGTH = 300
print('Number of sentences that exceed 300 characters')
print(L - len(df_train[~(df_train.sentence.apply(lambda x : len(x)) > MAX_LENGTH)]))

# print("Number of tweets : ", len(df_train))
# df_train['sentence'] = df_train['sentence'].map(lambda x: max_text_length(x, 300))
df_train = df_train[~(df_train.sentence.apply(lambda x : len(x)) > MAX_LENGTH)]

df_train

Number of sentences that exceed 300 characters
3103


,sentence,wikidata,label
0,juan antonio ramos sánchez born 18 august 1976...,place of birth biên hòa,1
1,born into the prominent borgia family in xàtiv...,place of birth Xàtiva,0
2,virat kohli was born on 5 november 1988 in del...,place of birth Delhi,0
3,"born in manchester, wellens started his career...",place of birth kikuka,1
4,"du ping , born 28 february 1978, in shenyang i...",place of birth yokohama,1
...,...,...,...
2994,tyutchev was born into a russian noble family ...,place of birth budapest,1
2995,"the second child of gustav josef kokoschka, a ...",place of birth seoul,1
2996,"james edward day was born in jacksonville, ill...",place of birth Jacksonville,0
2997,federico franco was born in the city of asunci...,place of birth Asunción,0


In [17]:
df_train.to_csv("train.csv", index=False)
df_val.to_csv("validation.csv", index=False)
df_test.to_csv("test.csv", index=False)

In [18]:
T = pd.read_csv('train.csv')
T1, T2, T3= T.iloc[0, :].values
print(T1)
print(T2)
print(T3)

juan antonio ramos sánchez born 18 august 1976 in barcelona is a spanish taekwondo practitioner
place of birth biên hòa
1


## Make Dataset

In [19]:
from torch.utils.data import Dataset

class WikiDataset(Dataset):
    # read the tsv we make and initialize some parameters
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "validation", "test"]
        self.mode = mode
        self.df = pd.read_csv(mode + ".csv")
        self.len = len(self.df)
        self.tokenizer = tokenizer  # use BERT tokenizer
    
    # define a function that reutrn a training or testing data
    def __getitem__(self, idx):
        if self.mode == "test" or self.mode == "validation":
            sentence, wikidata = self.df.iloc[idx, :2].values
            label_tensor = None
        else:
            sentence, wikidata, label_id = self.df.iloc[idx, :].values
            label_tensor = torch.tensor(label_id)
            
        # BERT tokens
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(sentence)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)
        
        tokens_b = self.tokenizer.tokenize(wikidata)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a
        
        # convert hole token sequence into index sequence
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # segments_tensor
        segments_tensor = torch.tensor([0] * len_a + [1] * len_b, dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len


### Make trainset

In [20]:
# Dataset
trainset = WikiDataset("train", tokenizer=tokenizer)

In [21]:
# select first sample
sample_idx = 0

# compare with the original document
sentence_0, wikidata_0, label_0 = trainset.df.iloc[sample_idx].values

# use the Dataset we built to extract the transformed id tensors
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

# convert the tokens_tensor to original document
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

print(f"""[original document]
sentence  ：{sentence_0}
wikidata  ：{wikidata_0}
label ：{label_0}

--------------------

[ tensors return by Dataset ]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

label_tensor   ：{label_tensor}

--------------------

[tokens_tensors word pieces]
{combined_text}
""")

[original document]
sentence  ：juan antonio ramos sánchez born 18 august 1976 in barcelona is a spanish taekwondo practitioner
wikidata  ：place of birth biên hòa
label ：1

--------------------

[ tensors return by Dataset ]
tokens_tensor  ：tensor([  101,   179,  8734, 22904, 11153,  1186, 26084,  2155,   188,  4881,
         4386,  1584,  1255,  1407, 12686, 12909,  1204,  2402,  1107,  2927,
        18389,  7637,  1110,   170,  8492,  2944, 27629,  4820, 16732,  2572,
        22351,   102,  1282,  1104,  3485, 16516, 24559,  1179,   177, 20142,
         1161,   102])

segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

label_tensor   ：1

--------------------

[tokens_tensors word pieces]
[CLS]j##uanant##oni##oram##oss##án##che##zborn18au##gus##t1976inbar##cel##onaisaspan##ishta##ek##won##dopractitioner[SEP]placeofbirthbi##ê##nh##ò##a[SEP]



## DataLoader

In [22]:
"""
Create a DataLoader that can return a mini-batch each time
This DataLoader works with the 'WikiDataset' we define previously
we need 4 tensors when training a BERT model：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# The input samples of this function is a list,
# every element in it is a sample return by the 'WikiDataset'

# Every sample contains 3 tensors : 
# - tokens_tensor
# - segments_tensor
# - label_tensor

# It will procecss zero padding on the first two tensors,
# then create a masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # with labels or not
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pading
    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)
    
    # attention masks, 
    # set the locations that are not zero padding tokens_tensors to 1 in order to let bert only focus on those tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

### trainloader

In [23]:
# Initialize a DataLoader
# use `collate_fn` to combine list of samples into a mini-batch
BATCH_SIZE = 24
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

In [24]:
data = next(iter(trainloader))

tokens_tensors, segments_tensors, masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([24, 77]) 
tensor([[  101,   179,  8734,  ...,     0,     0,     0],
        [  101,  1255,  1154,  ...,     0,     0,     0],
        [  101,   191,  5132,  ...,     0,     0,     0],
        ...,
        [  101,  1177, 27008,  ...,     0,     0,     0],
        [  101,  1113,   179,  ...,     0,     0,     0],
        [  101,   185,  1233,  ...,     0,     0,     0]])
------------------------
segments_tensors.shape = torch.Size([24, 77])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
------------------------
masks_tensors.shape    = torch.Size([24, 77])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
------------

## Import Pretrained BERT Model

In [25]:
# n_class = 2
from transformers import BertForSequenceClassification

PRETRAINED_MODEL_NAME = "bert-base-cased"
NUM_LABELS = 2

model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

clear_output()

print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=2, bias=True)


In [26]:
model.config

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

## Prediction

In [27]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        for data in dataloader:
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # first 3 tensors are tokens, segments and masks 
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # calculate accuracy when training
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # record current batch
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    

In [28]:
# run the model on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
# _, acc = get_predictions(model, trainloader, compute_acc=True)
# print("classification acc:", acc)

device: cuda:0


In [29]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
     
model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
number of parameters of whole model：{sum(p.numel() for p in model_params)}
number of parameters of the linear classifier：{sum(p.numel() for p in clf_params)}
""")


number of parameters of whole model：108311810
number of parameters of the linear classifier：1538



## Start training

In [30]:
%%time

# train model
model.train()

# using Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


EPOCHS = 3    # number of epochs
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]

        # set the gradients to zero
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()


        # record batch loss
        running_loss += loss.item()
        
    # calculate accuracy
    _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))

[epoch 1] loss: 24.757, acc: 0.993
[epoch 2] loss: 1.743, acc: 0.999
[epoch 3] loss: 1.411, acc: 1.000
Wall time: 3min 49s


In [41]:
# PATH = 'B:/Documents/2020五上/Wikidata/Wikipedia_Wikidata_Alignment/model_place_of_birth'
# torch.save(model, PATH)

## Validation

In [32]:
valiset = WikiDataset("validation", tokenizer=tokenizer)

In [33]:
valiloader = DataLoader(valiset, batch_size=32, collate_fn=create_mini_batch)

with torch.no_grad():
    validations = get_predictions(model, valiloader)
torch.cuda.empty_cache()

In [34]:
validations_numpy = validations.cpu().clone().numpy()
validations_numpy = validations_numpy.reshape((3000, 1))

In [35]:
ids = np.arange(1,3001)
ids = ids.reshape((3000, 1))

In [36]:
validations_array = np.concatenate((ids, validations_numpy), axis=1)
validations_array

array([[   1,    1],
       [   2,    0],
       [   3,    0],
       ...,
       [2998,    1],
       [2999,    0],
       [3000,    0]], dtype=int64)

In [37]:
df_validations = pd.DataFrame(data = validations_array, columns=["id", "label"])
df_validations['label'] = df_validations['label'].map({0:0, 1:1})
# df_validations.to_csv('answer.txt', index = False)

### Calculate F1 score

In [38]:
from sklearn.metrics import f1_score

print("F1 : ",f1_score(df_validations['label'], df_val['label'],  average = 'weighted'))

F1 :  0.9976666793704395


In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
print(confusion_matrix(df_val['label'], df_validations['label']))

[[1500    0]
 [   7 1493]]
